In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
%run alzheimer.ipynb

In [ ]:
"""
# DEFINITIONS
TRAIN_DIR_PATH = '../Alzheimer_s Dataset/train'
TEST_DIR_PATH = '../Alzheimer_s Dataset/test'
RESULT_PATH = 'results_feat_extr'
SEED = 42
DATASET_PERCENTAGE = 1
VALIDATION_PERCENTAGE = 0
CLASSES = ['NonDemented','VeryMildDemented','MildDemented','ModerateDemented']
VERBOSE = {
    'ANALYSIS': False,
    'LOADING': False,
    'SPLITTING': False,
    'EXECUTION': False,
    'EVALUATION': False
}
"""

In [ ]:
def prepare_dataset_feat_extr_resize_to_model(x, model_input_shape):
    """ Redefine as dimensões da entrada para ser compatível ao modelo da extração de características. 

    Retorna o tamanho da entrada e a distribuição das classes 

    Args:
        x: entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada redimensionada
    """

    X = []
    for i in range(0, x.shape[0]):
        # A imagem na escala de cinza é replicada para os 3 canais de cor, pois os modelos de extração de características esperam imagens coloridas.
        img = Image.fromarray(x[i], mode="RGB")
        X.append(np.array(img.resize(size=model_input_shape)))
    return np.array(X)


def prepare_dataset_feat_extr_input(x, input_shape, model_input_shape):
    """ Prepara a entrada para a extração de características

    Reordenadas as dimensões da entrada e ajusta o seu tamanho para ser compatível ao modelo da extração de características.

    Args:
        x: entrada
        input_shape: dimensões da entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada preparada para a extração de características
    """

    x_channel_formatted, _ = prepare_dataset_channel_position(
        x.astype('float32'), input_shape)
    return prepare_dataset_feat_extr_resize_to_model(x_channel_formatted, model_input_shape)


def prepare_dataset_feat_extr(x, y, input_shape, model_input_shape):
    """ Prepara o conjunto de dados para a extração de características

    Reordenadas as dimensões da entrada e ajusta o seu tamanho para ser compatível ao modelo da extração de características.

    Args:
        x: entrada
        y: saída
        input_shape: dimensões da entrada
        model_input_shape: dimensões esperadas pelo modelo de extração de características
        
    Returns:
        Entrada preparada para a extração de características,
        Saída preparada para a extração de características
    """

    x_prepared = prepare_dataset_feat_extr_input(
        x, input_shape, model_input_shape)
        
    # A saída é inalterada
    return x_prepared, y

In [ ]:
def extract_feature(model, x):
    """ Transforma a entrada em um conjunto de características utilizando o modelo

    Faz a previsão da entrada pelo modelo da extração de características. A saída gerada são as características extraídas.

    Args:
        model: modelo de extração de característica
        x: entrada
      
    Returns:
        Características extraídas do modelo
    """

    prediction = np.array(model.predict(x))
    return np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

In [ ]:
def feature_extraction(model_and_parameter):
    """ Realiza a extração de características do modelo

    Faz a leitura do conjunto de dados e extrai suas características.

    Args:
        model_and_parameter: variável que contém tanto o modelo como as dimensões da entrada esperada por ele
      
    Returns:
        Entrada de treinamento extraída,
        Saída de treinamento extraída,
        Entrada de teste extraída,
        Saída de teste extraída    
    """
    
    model, model_input_shape = model_and_parameter()

    input_shape, _ = analyse_dataset()
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)
    x_train, _, y_train, _ = split_dataset(x_trainval, y_trainval)

    x_train_prepared, y_train_prepared = prepare_dataset_feat_extr(
        x_train, y_train, input_shape, model_input_shape)
    x_test_prepared, y_test_prepared = prepare_dataset_feat_extr(
        x_test, y_test, input_shape, model_input_shape)

    x_train_extracted = extract_feature(model, x_train_prepared)
    x_test_extracted = extract_feature(model, x_test_prepared)

    return x_train_extracted, y_train_prepared, x_test_extracted, y_test_prepared


In [ ]:
def extract_and_evaluate(model_and_parameter, classifier):
    """ Faz a extração e classificação do conjunto de dados

    Faz a extração de característica e a usa como entrada do classificador. O resultado produzido pelo classificador é o resultado final.

    Args:
        model_and_parameter: variável que contém tanto o modelo como as dimensões da entrada esperada por ele
        classifier: classificador

    Returns:
        Acurácia 
    """
    
    x_train, y_train, x_test, y_test = feature_extraction(model_and_parameter)
    acc , _ = classifier(x_train, y_train, x_test, y_test)
    return acc,